In [1]:
! pip install sentencepiece==0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 34.1 MB/s 


In [2]:
!pip install transformers==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 563 kB 29.1 MB/s 
     |████████████████████████████████| 5.6 MB 25.1 MB/s 
     |████████████████████████████████| 880 kB 63.0 MB/s 
     |████████████████████████████████| 132 kB 71.3 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 9.1 MB 60.0 MB/s 
     |████████████████████████████████| 140 kB 69.3 MB/s 
     |████████████████████████████████| 127 kB 76.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b4f720dcaaafe5d917b776cb1d2f1903e5d177521c12ac16ab0398e5743bef90
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalle

In [3]:
! pip install google

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import json 
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [5]:
#  t5-small Pretrained Model

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [6]:
import bs4
import requests
from bs4 import BeautifulSoup 
import numpy as np
import googlesearch
import pandas as pd

In [34]:
def search(query):
  googleSearch = googlesearch.search(query, num=20, lang='en')
  notEnough = True
  all_texts = []
  for link in googleSearch:
    if len(all_texts) < 10 :
      txt = []
      reqs = requests.get(link)
      soup = BeautifulSoup(reqs.text, 'html.parser')
      for p in soup.find_all('p'):
        for childp in p:
          cont = str(childp.string)
          if len(cont) > 20:
            txt.append(cont)
      txt = txt[int(len(txt)/10):int(len(txt)*3/4)]
      txt = " ".join(txt)
      if len(txt.split(' ')) > 500:
        txt = txt.split(' ')
        txt = txt[:500]
        txt = " ".join(txt)
        all_texts.append(txt)
      elif len(txt.split(' ')) < 175:
        pass;
      else:
        all_texts.append(txt)
    else :
      break;

  return all_texts


In [8]:
def summarize_1(search_results):
  sum_texts = []
  for i in range(len(search_results)):
    to_sum_Text = "summarize: " + search_results[i]

    tokenized_text = tokenizer.encode(to_sum_Text, return_tensors="pt").to(device)


    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=50,
                                        max_length=100,
                                        early_stopping=True)

    sum_texts.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
  return sum_texts


In [9]:
def summarize_2(summary_results):
  summary_results = ", ".join(summary_results)

  to_sum_Text = "summarize: " + summary_results

  tokenized_text = tokenizer.encode(to_sum_Text, return_tensors="pt").to(device)


  # summmarize 
  summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=2,
                                      min_length=75,
                                      max_length=125,
                                      early_stopping=True)

  sum_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return sum_text

In [10]:
#   &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

In [11]:
#   &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

In [35]:
search_results = search('Harry Poter')

In [36]:
for i,pa in enumerate(search_results):
  length = len(pa.split(' '))
  print(f'\nParagraph {i+1}: --------    {length} Words.\n{pa}')


Paragraph 1: --------    500 Words.
 describes Harry's second year at Hogwarts. Students are attacked and petrified by an unknown creature; wizards of Muggle parentage are the primary targets. The attacks appear related to the  , a fifty-year-old mystery at the school. Harry discovers an ability to speak the snake language  , which he learns is rare and associated with the  . When Hermione is attacked, Harry and Ron uncover the chamber's secrets and enter it. Harry discovers that the chamber was opened by Ron's younger sister,  , who was possessed by an old diary in her belongings. The memory of  , Voldemort's younger self, resided inside the diary and unleashed the  , an ancient monster that kills those who make direct eye contact. Harry draws the  , slays the basilisk and destroys the diary.
 Harry Potter and the Prisoner of Azkaban , Harry learns that he is targeted by  , an escaped convict who allegedly assisted in his parents' murder. As Harry struggles with his reaction to the  

In [37]:
# preprocessing texts
for i in range(len(search_results)):
  search_results[i] = search_results[i].strip().replace("\n","")

In [38]:
# summarizing all paragrafes
summary_results = summarize_1(search_results)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1292: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


In [39]:
for i,pa in enumerate(summary_results):
  length = len(pa.split(' '))
  print(f'\nSummarized Paragraph {i+1}: --------    {length} Words.\n{pa}')


Summarized Paragraph 1: --------    48 Words.
the attacks appear related to the, a fifty-year-old mystery at the school. Harry discovers an ability to speak the snake language, which he learns is rare and associated with the wizards of Muggle parentage. when Hermione is attacked, Harry and Ron uncover the chamber's secrets and enter it.

Summarized Paragraph 2: --------    50 Words.
this video features some new LEGO Harry Potter Hogwarts sets for you to get excited about. check out glimpses of some sets you may recognise, such as the Chamber of Secrets and the Whomping Willow. if you have someone in your life who doesn’t know their Muggles from their Mugwumps?

Summarized Paragraph 3: --------    58 Words.
rowling was hesitant to sell the rights because she "didn't want to give them control over the rest of the story" the film would have enabled Warner Bros. to make non-author-written sequels. he says she had no role in choosing directors for the films, citing his work on other family

In [40]:
final_text = summarize_2(summary_results)

In [41]:
length = len(final_text.split(' '))
final_text = final_text.split('.')
print(f'\nFinal Text: --------    {length} Words.\n')
for i in range(len(final_text)):
  print(final_text[i]+'\n')


Final Text: --------    56 Words.

new LEGO Harry Potter Hogwarts sets include the chamber of secrets and the Whomping Willow

 the film would have enabled Warner Bros

 to make non-author-written sequels - citing his work on other family films such as as influences for their decision

 if you have someone in your life who doesn’t know their Muggles from their Mugwumps?

